# Advanced Features
## Comparison of fit results

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
try:
    import KiMoPack.plot_func as pf
except:
    print("General installation did not work, try to import from the same folder as a workaround")
    import plot_func as pf
%matplotlib inline

## 1) Load Projects

In this example it is demonstrated how to work with saved *hdf5* projects. Upon saving a project all parameters of the analysis are dumped to the *hdf5* file. This includes the standard corrected (background, arrival time, scattercut, ...) TA data as well as the fit results and parameter. 

Herein, the TA projects of **Ru-dppz** collected in three solvents, namely dichloromethane (DCM), acetonitrile (ACN) and water (H$_2$O) at 400 nm excitation are loaded and compared. Firstly, the master project (here in ACN) is loaded to the ```ta_ACN``` object. Secondly all *hdf5* projects from a specified folder (including the data in DCM and H$_2$O) are loaded into the object ```compare_projects```. For loading the comparison projects the function ```GUI_open``` is employed. You can either read all *hdf5* projects from a folder (```project_list='all'```) or you can select single projects from the file explorer (```project_list='gui'```).

In [ ]:
# initialize ta_ACN object including the TA data in ACN
ta_ACN = pf.TA(filename='TA_Ru-dppz_400nm_ACN_paral.hdf5',
               path=os.path.join('Data', 'Compare', 'Master'))

# initialize an object including the TA data in DCM and H2O
compare_projects=pf.GUI_open(project_list='all',
                             path=os.path.join('Data', 'Compare'))

# plot TA spectra of the master project
ta_ACN.Plot_fit_output(title='Master project, ACN, 400 nm', plotting=[3])

## 2) Compare transient spectra

Upon loading the TA data of several projects, the user can compare the transient spectra by plotting multiple spectra into the same figure at given delay times (```rel_time```). Herein, the TA spectra of **Ru-dppz** obtained in different solvents, namely dichlorometahne (DCM), acetonitrile (ACN) and water (H$_2$O) are compared.

### General settings

- For a better comparison the data of each project can be normalized to the master object (in this example ```ta_ACN```). For the normalization a normalization range is defined by the lower and upper limits of deay times and probe wavelengths (*e.g.* ```norm_window=[0.5,0.7,420,470]```). 

- In order to be able to compare the individual data sets well with each other, the use of a highly diverging colormap is recommended. Herein the colormap *Accent* is used. For more available maps see: https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
norm_window = [0.5,0.7,420,470]  # norm window
ta_ACN.cmap=pf.cm.Accent         # use a diverge colormap for comparison

### 2a) Compare transient spectra

In [ ]:
for delay_time in [1,10,100]:                    # plot spectra for each selected delay time
    ta_ACN.Compare_at_time(fitted=False,         # compare pre-processed data
                       other=compare_projects,   # list of projects to compare
                       rel_time=[delay_time],    # selected deay times to compare
                       norm_window=norm_window)  # set norm window

### 2b) Compare transient and external spectra

To compare the transient spectra at a certain delay time to an external spectrum (*e.g.* spectro-electro-chemistry or steady state absorption data), the user can pass a dataframe with such spectra to the ```Compare_at_time```function. Herein, the absorption data of electrochemically unmodified **Ru-dppz** and its singly reduced form are loaded into a Pandas DataFrame with the wavelength column as index. In the example the scaled difference spectrum of the reduced and unreduced complex is plotted for comparison to the TA spectra of **Ru-dppz** in ACN and H$_2$O obtained at a delay time of 1 ps.

In [ ]:
# define name of external spectra
spectra_name = 'UVvis_SEC_Rudppz_ACN.dat'

# define path of external spectra
spectra_path = os.path.join(os.getcwd(), 'Data', 'Compare')

# create dataframe of external spectra
SEC_df = pd.read_csv(os.path.join(spectra_path, spectra_name), index_col=0, sep="\t", header=0)
diff_spectrum=(SEC_df['red']-SEC_df['ocp'])*0.05   # create difference spectrum
diff_spectrum.name='Rudppz_ACN - difference'       # give it a name for the plot

ta_ACN.Compare_at_time(fitted=False,
                       rel_time=1.0,               # selected delay time
                       other=compare_projects,     # list of projects to compare
                       spectra=diff_spectrum,      # external spectra to compare
                       norm_window=norm_window)    # set norm window

### 2c) Compare kinetic traces

The kinetic traces of several projects at a given probe wavelength (```rel_wave```) can be plotted into the same figure by the ```Comapre_at_wave```function. This allows to compare the kinetics at various conditions (*e.g.* pump intensity, quencher concentration, solvent). A normalization window can be given at which all the plotted curves are normalized to. This window does not have to be in the plotted region.In this example the TA kinetics of the excited state absorption at 340 and 580 nm and the ground state bleach minimum at 440 nm in DCM, ACN and H$_2$O are compared.

In [ ]:
ta_ACN.cmap=pf.cm.Accent
for nm in [340,440,580]:                            # plot kinetics at each selected wavelengths
    ta_ACN.timelimits=[-0.5,1500]                   # set timelimits of the plot
    ta_ACN.Compare_at_wave(fitted=False,            # plot preprocessed data
                           other=compare_projects,  # list of projects to compare
                           rel_wave=nm,             # selected wavelengths to compare
                           norm_window=norm_window) # norm window